In [90]:
import nltk
import re
import spacy
import pandas as pd
import translate
from config import PATH

In [ ]:
# import ssl

# # RUN THIS CELL TO GET THE RIGHT FILES FOR → nltk
# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()

# nltk.download('punkt')
# nltk.download('stopwords')

In [66]:
# define the path to the file
path = "wos/subtitles/silicon_valley/season_1/silicon_valley_1x01_en.srt"

with open(f"{PATH}/{path}", "r") as file:
    subtitles = file.read()

In [67]:
pattern = r"[^a-zA-Z\s]"

clean_subtitles = re.sub(pattern, " ", subtitles)

In [68]:
words = nltk.word_tokenize(clean_subtitles.lower())
# words_df = pd.DataFrame(words, columns=["word"])
len(words)

4133

In [69]:
# FIRSTLY DOWNLOAD python -m spacy download en_core_web_sm
model = spacy.load("en_core_web_sm")

In [70]:
columns = ["word", "error"]
errors_df = pd.DataFrame([], columns=columns)

lemmas = []
for word in words:
    try:
        doc = model(word)
        for token in doc:
            lemma = token.lemma_
            lemmas.append(lemma)
    
    except Exception as ex:
        temp_df = pd.DataFrame([(word, ex)], columns=columns)
        errors_df = pd.concat([errors_df, temp_df], ignore_index=True)

In [71]:
#stopwords = nltk.corpus.stopwords.words("english")

In [94]:
lemmas_df = pd.DataFrame(lemmas, columns=["lemma"])
# filtered_lemmas_df = lemmas_df[~lemmas_df["lemma"].isin(stopwords)]

In [98]:
words_df = lemmas_df['lemma'].value_counts().reset_index()
words_df.columns = ['lemma', 'cnt']

In [99]:
def from_en_to_ru(word):
    translator = translate.Translator(to_lang="ru")
    word_ru = translator.translate(word)
    return word_ru

In [100]:
words_df['ru'] = words_df['lemma'].apply(from_en_to_ru)

In [101]:
ru_words_df = words_df[["cnt", "lemma", "ru"]].sort_values(by="cnt", ascending=False)

In [104]:
csv_path = f"{PATH}/wos/dictionary/silicon_valley/csv/silicon_valley_1x01_en_ru.csv"
ru_words_df.to_csv(csv_path, index=False)

: 